In [2]:
import spotipy, pandas
from spotipy.oauth2 import SpotifyOAuth

In [8]:
scope = 'playlist-modify-public playlist-modify-private'
red_url='http://localhost'

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(redirect_uri=red_url,scope=scope))